In [3]:
# Arreglar rutas en positives_24x24.txt
pos_path = "//home/romero/Escritorio/LBPDetectorBackend/validation/positives.txt"
fixed_pos_path = pos_path.replace(".txt", "_fixed.txt")

with open(pos_path, "r") as fin, open(fixed_pos_path, "w") as fout:
    for line in fin:
        parts = line.strip().split()
        filename = os.path.basename(parts[0])  # solo nombre del archivo
        fout.write(" ".join([filename] + parts[1:]) + "\n")

print("✅ Archivo positivo corregido:", fixed_pos_path)

# Arreglar rutas en bg.txt
bg_path = "/home/romero/Escritorio/LBPDetectorBackend/validation/bg.txt"
fixed_bg_path = bg_path.replace(".txt", "_fixed.txt")

with open(bg_path, "r") as fin, open(fixed_bg_path, "w") as fout:
    for line in fin:
        filename = os.path.basename(line.strip())
        fout.write(filename + "\n")

print("✅ Archivo negativo corregido:", fixed_bg_path)


✅ Archivo positivo corregido: //home/romero/Escritorio/LBPDetectorBackend/validation/positives_fixed.txt
✅ Archivo negativo corregido: /home/romero/Escritorio/LBPDetectorBackend/validation/bg_fixed.txt


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_score,
    recall_score
)

# === CONFIGURACIÓN ===
MODEL_PATH = "/home/romero/Escritorio/LBPDetectorBackend/classifier/cascade.xml"
VAL_IMAGES_DIR = "/home/romero/Escritorio/LBPDetectorBackend/validation/val_images"
POS_FILE = "/home/romero/Escritorio/LBPDetectorBackend/validation/positives_fixed.txt"
NEG_FILE = "/home/romero/Escritorio/LBPDetectorBackend/validation/bg_fixed.txt"

# === CARGAR CLASIFICADOR ===
classifier = cv2.CascadeClassifier(MODEL_PATH)
if classifier.empty():
    raise IOError("No se pudo cargar el clasificador")

# === CARGAR LISTAS DE IMÁGENES ===
def load_filenames(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
    return [line.strip().split()[0] for line in lines]

positive_images = load_filenames(POS_FILE)
negative_images = load_filenames(NEG_FILE)

# === EVALUACIÓN ===
y_true = []
y_pred = []

# Evaluar imágenes positivas (deben detectar rostro → 1)
for img_name in positive_images:
    img_path = os.path.join(VAL_IMAGES_DIR, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"No se pudo cargar: {img_path}")
        continue
    faces = classifier.detectMultiScale(img, scaleFactor=1.1, minNeighbors=3)
    y_true.append(1)
    y_pred.append(1 if len(faces) > 0 else 0)

# Evaluar imágenes negativas (no deben detectar rostro → 0)
for img_name in negative_images:
    img_path = os.path.join(VAL_IMAGES_DIR, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"No se pudo cargar: {img_path}")
        continue
    faces = classifier.detectMultiScale(img, scaleFactor=1.1, minNeighbors=3)
    y_true.append(0)
    y_pred.append(1 if len(faces) > 0 else 0)

# === RESULTADOS ===
print("\n--- Reporte de Clasificación ---")
print(classification_report(y_true, y_pred, target_names=["Negativo (No rostro)", "Positivo (Rostro)"]))

# === MATRIZ DE CONFUSIÓN ===
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No rostro", "Rostro"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusión")
plt.show()

# === MÉTRICAS ===
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)

if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
else:
    print("⚠️ Advertencia: Solo se detectó una clase en las predicciones.")
    specificity = 0

# === GRÁFICO DE MÉTRICAS ===
metrics = {
    "Precisión": precision,
    "Sensitividad": recall,
    "Especificidad": specificity
}

plt.figure(figsize=(6, 4))
plt.bar(metrics.keys(), metrics.values(), color=["blue", "green", "red"])
plt.ylim(0, 1)
plt.title("Métricas de Desempeño")
plt.ylabel("Valor")
plt.grid(axis='y')
plt.show()

# === DIAGNÓSTICO EXTRA ===
print("\nConteo de predicciones:")
print("→ Predichos como 'No rostro' (0):", y_pred.count(0))
print("→ Predichos como 'Rostro' (1):", y_pred.count(1))


No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00000.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00001.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00002.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00003.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00004.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00005.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00006.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00007.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00008.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorB

[ WARN:0@4.753] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00000.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4.757] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00001.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4.757] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00002.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4.757] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00003.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4.757] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00004.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4

No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00826.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00827.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00828.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00829.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00830.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00831.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00832.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00833.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00834.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorB

[ WARN:0@4.982] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00894.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4.988] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00895.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4.988] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00896.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4.988] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00897.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4.988] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_00898.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4

No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_01453.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_01454.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_01455.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_01456.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_01457.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_01458.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_01459.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_01460.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/images/img_01461.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorB

...

[ WARN:0@8.586] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05196.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8.588] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05197.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8.588] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05198.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8.588] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05199.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8.588] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05200.jpg'): can't open/read fi

No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05224.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05225.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05226.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05227.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05228.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05229.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05230.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05231.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_fl

[ WARN:0@8.824] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05891.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8.824] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05892.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8.825] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05893.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8.825] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05894.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8.830] global loadsave.cpp:275 findDecoder imread_('/home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05895.jpg'): can't open/read fi

No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05909.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05910.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05911.jpg
No se pudo cargar: /home/romero/Escritorio/LBPDetectorBackend/validation/val_images/negatives_flat/neg_05912.jpg

--- Reporte de Clasificación ---


ValueError: Number of classes, 0, does not match size of target_names, 2. Try specifying the labels parameter